# Four Set Mahjong Probabilities

This notebook tabulates hand probabilities for a teaching Mahjong variant with a limited number of types and hand size:
- **Tiles**: 120 tiles covering the standard numeric suits (bamboo, circles, characters), plus dragons (white, green, red); four copies of each tile.
- **Hand Size**: Players seek to complete a hand with 14 tiles, consisting of 4 sets of three (sequence or triplet; no quads) and 1 pair.
- **Calls**: As a side note, calls for sequences (_chii_) and triplets (_pon_) are allowed.

In traditional Mahjong, patterns of tiles in a completed hand are given point values based generally on their elegance and rarity: how do the rarities of those patterns change when we limit the types of tiles and the number of tiles in hand?

In [72]:
import math
import numpy as np
import pandas as pd

from itertools import product

In [73]:
# load pre-computed tile combination properties for numeric tiles
suited_df = pd.read_csv('./shanten_suuhai.csv', 
                        index_col='tile_int', 
                        dtype={'tile_vector': str})

print(suited_df.shape)
suited_df.sample(10)

(405350, 11)


,tile_vector,n_tiles,n_sets,n_triplets,n_sequences,n_blocks,n_pairs,max_pairs,n_koritsu,n_terminals,n_ways
tile_int,,,,,,,,,,,
133557778,102020310,9,1,1,0,2,2,3,1,1,2304
1111667788899,400002232,13,3,1,2,1,1,5,1,2,864
13335577788899,103020332,14,3,3,2,2,2,5,1,2,9216
1335566779999,102022204,13,3,1,2,1,1,5,1,2,5184
2444588,010310020,7,1,1,0,1,1,2,2,0,384
22233445667888,032212130,14,4,1,3,0,0,5,2,0,55296
12222333456688,143112020,14,3,2,2,2,2,4,0,1,9216
11233445557789,212230211,14,3,0,3,2,2,5,0,2,331776
133336888899,104001042,12,2,2,0,3,1,3,0,2,96


In [74]:
# load pre-computed tile combination properties for honor tiles
dragon_df = pd.read_csv('./shanten_jihai.csv', 
                        index_col='tile_int', 
                        dtype={'tile_vector': str})

# trim to only combinations that only contain dragons
no_winds = dragon_df['tile_vector'].apply(lambda x: x[:4]) == '0000'
dragon_df = dragon_df[no_winds]

print(dragon_df.shape)
dragon_df.sample(10)

(125, 7)


,tile_vector,n_tiles,n_triplets,n_pairs,n_koritsu,n_terminals,n_ways
tile_int,,,,,,,
56667777,0000134,8,2,0,2,3,16
555666777,0000333,9,3,0,0,3,64
66777,0000023,5,1,1,0,2,24
566667777,0000144,9,2,0,3,3,4
66,0000020,2,0,1,0,1,6
6,0000010,1,0,0,1,1,4
555566677,0000432,9,2,1,1,3,24
5555666777,0000433,10,3,0,1,3,16
556666,0000240,6,1,1,1,2,6


In [60]:
def vector_to_int(t_vector):
    t_int = ''
    for i, cnt in zip(np.arange(1,len(t_vector)+1),t_vector):
        t_int += cnt * str(i)
    if t_int:
        return int(t_int)
    else:
        return 0

def int_to_vector(t_int, n_types=9):
    t_vector = np.zeros(n_types, dtype=int)
    t_int = str(t_int)
    for i in t_int:
        t_vector[int(i)-1] += 1
    return t_vector

def string_to_vector(t_str):
    t_vector = np.zeros(9,dtype=int)
    for i in range(len(t_str)):
        t_vector[i] = int(t_str[i])
    return t_vector

## General Probabilities
- How many possible hands are there?
- How many of those hands form a winning combination? (Tenhou/Chiihou equivalent)
  - Standard hands: 4 sets + 1 pair
  - Chiitoi hands: 7 pairs (which can overlap with standard hands as ryanpeikou)

In [5]:
### How many possible hands are there, winning or otherwise?
n_tiles = 30 * 4
hand_size = 14
total_hands = math.comb(n_tiles,hand_size)

print(total_hands)

669413654240461560


In [6]:
### How many possible winning hands are there?
### Standard Hands
suited_complete = suited_df.query('(3 * n_sets + 2 * n_pairs == n_tiles) & n_pairs <= 1')
suited_complete_ways = suited_complete.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()
suited_complete_ways

,n_tiles,n_sets,n_pairs,n_ways
0,0,0,0,1
1,2,0,1,54
2,3,1,0,484
3,5,1,1,19200
4,6,2,0,65272
5,8,2,1,1748756
6,9,3,0,2742868
7,11,3,1,47037380
8,12,4,0,40399783
9,14,4,1,440593684


In [7]:
dragon_complete = dragon_df.query('(3 * n_triplets + 2 * n_pairs == n_tiles) & (n_pairs <= 1)')
dragon_complete_ways = dragon_complete.groupby(['n_tiles', 'n_triplets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()
dragon_complete_ways = dragon_complete_ways.rename(columns={'n_triplets':'n_sets'})
dragon_complete_ways

,n_tiles,n_sets,n_pairs,n_ways
0,0,0,0,1
1,2,0,1,18
2,3,1,0,12
3,5,1,1,144
4,6,2,0,48
5,8,2,1,288
6,9,3,0,64


In [8]:
standard_winning_hands = 0
for sou_idx in suited_complete_ways.index:
    sou_part = suited_complete_ways.loc[sou_idx]

    pin_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]

        man_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']-pin_part['n_tiles']]
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hon_ways = dragon_complete_ways[dragon_complete_ways['n_tiles'] == 14-sou_part['n_tiles']-pin_part['n_tiles']-man_part['n_tiles']]
            for hon_idx in hon_ways.index:
                hon_part = hon_ways.loc[hon_idx]
                hand = sou_part + pin_part + man_part + hon_part

                if hand['n_pairs'] == 1:
                    standard_winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

print(standard_winning_hands)
print(f"proportion: {standard_winning_hands/total_hands:0.3e}; 1 in {total_hands/standard_winning_hands:.0f}")

8840918606082
proportion: 1.321e-05; 1 in 75718


In [9]:
### Chiitoitsu / Seven Pairs
suited_pairs = suited_df.query('2 * max_pairs == n_tiles')
suited_pairs_ways = suited_pairs.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()
suited_pairs_ways

,n_tiles,n_sets,n_pairs,n_ways
0,0,0,0,1
1,2,0,1,54
2,4,0,2,1296
3,6,0,3,16632
4,6,2,0,1512
5,8,0,4,116640
6,8,2,1,46656
7,10,0,5,396576
8,10,2,2,583200
9,12,0,6,466560


In [10]:
dragon_pairs = dragon_df.query('2 * n_pairs == n_tiles')
dragon_pairs_ways = dragon_pairs.groupby(['n_tiles', 'n_triplets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()
dragon_pairs_ways = dragon_pairs_ways.rename(columns={'n_triplets':'n_sets'})
dragon_pairs_ways

,n_tiles,n_sets,n_pairs,n_ways
0,0,0,0,1
1,2,0,1,18
2,4,0,2,108
3,6,0,3,216


In [11]:
chiitoi_winning_hands = 0
for sou_idx in suited_pairs_ways.index:
    sou_part = suited_pairs_ways.loc[sou_idx]

    pin_ways = suited_pairs_ways[suited_pairs_ways['n_tiles'] <= 14-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]

        man_ways = suited_pairs_ways[suited_pairs_ways['n_tiles'] <= 14-sou_part['n_tiles']-pin_part['n_tiles']]
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hon_ways = dragon_pairs_ways[dragon_pairs_ways['n_tiles'] == 14-sou_part['n_tiles']-pin_part['n_tiles']-man_part['n_tiles']]
            for hon_idx in hon_ways.index:
                hon_part = hon_ways.loc[hon_idx]
                hand = sou_part + pin_part + man_part + hon_part

                if hand['n_sets'] != 4: # exclude hands that can form a standard hand
                    chiitoi_winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

print(chiitoi_winning_hands)
print(f"proportion: {chiitoi_winning_hands/total_hands:0.3e}; 1 in {total_hands/chiitoi_winning_hands:.0f}")
print(f"ratio vs standard hands: 1 to {standard_winning_hands/chiitoi_winning_hands:.1f}")

568785162240
proportion: 8.497e-07; 1 in 1176918
ratio vs standard hands: 1 to 15.5


In [12]:
### Overall winning chances
total_winning_hands = standard_winning_hands + chiitoi_winning_hands

print(total_winning_hands)
print(f"proportion: {total_winning_hands/total_hands:0.3e}; 1 in {total_hands/total_winning_hands:.0f}")

9409703768322
proportion: 1.406e-05; 1 in 71141


## Specific Hand Type Proportions
- **Terminals and Honors**
  - All Simples (_tanyao_): only numeric tiles from 2-8
  - Included Terminals and Honors (_chanta_): each set and the pair includes a 1, 9, or dragon
  - Included Terminals (_junchan_): each set and the pair includes a 1 or 9; no dragons
  - All Terminals and Honors (_honroutou_): each set and the pair consists of only 1s, 9s, or dragons
  - All Terminals (_chinroutou_): each set and the pair consists of only 1s or 9s; no dragons
- **Set Consistency**
  - All Sequences (_pinfu_-like): three sequences and a pair
  - All Triplets (_toitoi_; _sanankou_-like): three triplets and a pair
- **Dragon Triplets**
  - Dragon Triplet (_yakuhai_): triplet of dragons
  - 2x Dragon Triplet: two triplets of dragons
  - Small Three Dragons (_shousangen_): two triplets of dragons + pair of third
  - Big Three Dragons (_daisangen_): three triplets of dragons
- **Single Numeric Suit**
  - Half Flush (_honitsu_): all tiles are of a single numeric suit (bamboo, circles) or dragons
  - Full Flush (_chinitsu_): all tiles are of a single numeric suit; no dragons
- **Other Set Patterns**
  - Two Identical Sequences (_iipeikou_-like): two identical sequences, including same suit
  - Twice Two Identical Sequences (_ryanpeikou_-like): two instances of Two Identical Sequences
  - Full Straight (_ikkitsuukan_): sequences of 1-9 in a single suit
  - Three Similar Sequences (_sanshoku doujun_): three sequences with the same numbers, one in eacn suit
  - Three Similar Triplets (_sanshoku doukou_): three triplets with the same numbers, one in each suit

In [13]:
# defining sets for assembling winning combinations
sequences = [int_to_vector(123), int_to_vector(234), int_to_vector(345), int_to_vector(456),
             int_to_vector(567), int_to_vector(678), int_to_vector(789), np.zeros(9,dtype=int)]
triplets  = [int_to_vector(111), int_to_vector(222), int_to_vector(333), int_to_vector(444), int_to_vector(555),
             int_to_vector(666), int_to_vector(777), int_to_vector(888), int_to_vector(999), np.zeros(9,dtype=int)]
sets      = sequences[:-1] + triplets

pairs = [int_to_vector(11), int_to_vector(22), int_to_vector(33), int_to_vector(44), int_to_vector(55),
         int_to_vector(66), int_to_vector(77), int_to_vector(88), int_to_vector(99), np.zeros(9,dtype=int)]

terminal_sequences = [int_to_vector(123), int_to_vector(789), np.zeros(9,dtype=int)]
terminal_triplets  = [int_to_vector(111), int_to_vector(999), np.zeros(9,dtype=int)]
terminal_sets      = terminal_sequences[:-1] + terminal_triplets
terminal_pairs     = [int_to_vector(11),  int_to_vector(99),  np.zeros(9,dtype=int)]

In [14]:
def assemble_from_groups(*args):
    test_groups = product(*args)

    valid_groups = []
    for test_group in test_groups:
        test_vector = np.array(test_group).sum(axis=0)
        if (test_vector <= 4).sum() == test_vector.size:
            valid_groups.append(vector_to_int(test_vector))
    valid_groups = np.unique(np.array(valid_groups))
    
    return valid_groups

### Terminals and Honors

In [15]:
### All Simples
# standard hands
simple_complete = suited_complete.query('n_terminals == 0')
simple_complete_ways = simple_complete.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

winning_hands = 0
for sou_idx in simple_complete_ways.index:
    sou_part = simple_complete_ways.loc[sou_idx]

    pin_ways = simple_complete_ways[simple_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]

        man_ways = simple_complete_ways[simple_complete_ways['n_tiles'] == 14-sou_part['n_tiles']-pin_part['n_tiles']]
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hand = sou_part + pin_part + man_part

            if hand['n_pairs'] == 1:
                winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways']

# chiitoi hands
simple_pairs = suited_df.query('2 * max_pairs == n_tiles & n_terminals == 0')
simple_pairs_ways = simple_pairs.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

for sou_idx in simple_pairs_ways.index:
    sou_part = simple_pairs_ways.loc[sou_idx]

    pin_ways = simple_pairs_ways[simple_pairs_ways['n_tiles'] <= 14-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]

        man_ways = simple_pairs_ways[simple_pairs_ways['n_tiles'] == 14-sou_part['n_tiles']-pin_part['n_tiles']]
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hand = sou_part + pin_part + man_part

            if hand['n_sets'] != 4: # exclude hands that can form a standard hand
                winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways']

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.2f}")

931920119892
proportion: 0.0990382; 1 in 10.10


In [16]:
### Included Terminals and Honors
valid_groups = assemble_from_groups(terminal_sets, terminal_sets, terminal_sets, terminal_sets, terminal_pairs)

terminal_complete = suited_complete.loc[valid_groups,:]
terminal_complete_ways = terminal_complete.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

winning_hands = 0
for hon_idx in dragon_complete_ways.index[1:]: # must include honor tiles
    hon_part = dragon_complete_ways.loc[hon_idx]

    sou_ways = terminal_complete_ways[terminal_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']]
    for sou_idx in terminal_complete_ways.index:
        sou_part = terminal_complete_ways.loc[sou_idx]

        pin_ways = terminal_complete_ways[terminal_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']-sou_part['n_tiles']]
        for pin_idx in pin_ways.index:
            pin_part = pin_ways.loc[pin_idx]
            
            man_ways = terminal_complete_ways[terminal_complete_ways['n_tiles'] == 14-hon_part['n_tiles']-sou_part['n_tiles']-pin_part['n_tiles']] 
            for man_idx in man_ways.index:
                man_part = man_ways.loc[man_idx]

                hand = sou_part + pin_part + man_part + hon_part

                if hand['n_pairs'] == 1: 
                    winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

winning_hands -= (math.comb(9, 4) * 5 - math.comb(6, 4) * 2) * 4 ** 4 * 6 # exclude "All Terminals and Honors" hands

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.0f}")

9744752364
proportion: 0.0010356; 1 in 966


In [17]:
### Included Terminals
winning_hands = 0
for sou_idx in terminal_complete_ways.index:
    sou_part = terminal_complete_ways.loc[sou_idx]

    pin_ways = terminal_complete_ways[terminal_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]
        
        man_ways = terminal_complete_ways[terminal_complete_ways['n_tiles'] == 14-sou_part['n_tiles']-pin_part['n_tiles']] 
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hand = sou_part + pin_part + man_part

            if hand['n_pairs'] == 1: 
                winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways']

winning_hands -= math.comb(6, 4) * 2 * 4 ** 4 * 6 # exclude "All Terminals" hands

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.0f}")


6368641140
proportion: 0.0006768; 1 in 1478


In [18]:
### All Terminals and Honors
winning_hands  = math.comb(9, 4) * 5 # select four sets and pair
winning_hands -= math.comb(6, 4) * 2 # exclude "All Terminals" hands
winning_hands *= 4 ** 4 * 6 # select tiles within each group

winning_hands += 6 ** 7 # chiitoi pattern hands

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.3e}; 1 in {total_winning_hands/winning_hands:.0f}")

1201536
proportion: 1.277e-07; 1 in 7831396


In [19]:
### All Terminals
winning_hands  = math.comb(6, 4) * 2 # select four sets and pair
winning_hands *= 4 ** 4 * 6 # select tiles within each group

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.3e}; 1 in {total_winning_hands/winning_hands:.0f}")

46080
proportion: 4.897e-09; 1 in 204203641


### Set Consistency

In [20]:
# All Sequences
valid_groups = assemble_from_groups(sequences, sequences, sequences, sequences, pairs)

sequences_complete = suited_complete.loc[valid_groups,:]
sequences_complete_ways = sequences_complete.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

winning_hands = 0
for hon_idx in range(2): # include honor tiles as possible
    hon_part = dragon_complete_ways.loc[hon_idx]

    sou_ways = sequences_complete_ways[sequences_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']]
    for sou_idx in sou_ways.index:
        sou_part = sequences_complete_ways.loc[sou_idx]

        pin_ways = sequences_complete_ways[sequences_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']-sou_part['n_tiles']]
        for pin_idx in pin_ways.index:
            pin_part = pin_ways.loc[pin_idx]
            
            man_ways = sequences_complete_ways[sequences_complete_ways['n_tiles'] == 14-hon_part['n_tiles']-sou_part['n_tiles']-pin_part['n_tiles']] 
            for man_idx in man_ways.index:
                man_part = man_ways.loc[man_idx]

                hand = sou_part + pin_part + man_part + hon_part

                if hand['n_pairs'] == 1: 
                    winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.2f}")

5604375406650
proportion: 0.5955953; 1 in 1.68


In [21]:
# All Sequences; no dragons
winning_hands = 0
for sou_idx in sequences_complete_ways.index:
    sou_part = sequences_complete_ways.loc[sou_idx]

    pin_ways = sequences_complete_ways[sequences_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]
        
        man_ways = sequences_complete_ways[sequences_complete_ways['n_tiles'] == 14-sou_part['n_tiles']-pin_part['n_tiles']] 
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hand = sou_part + pin_part + man_part

            if hand['n_pairs'] == 1: 
                winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways']

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.2f}")

4778454510672
proportion: 0.5078220; 1 in 1.97


In [22]:
# All Triplets
winning_hands  = math.comb(30,4) * 26 # select four sets and a pair
winning_hands *= 4 ** 4 * 6 # select tiles within each group

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.0f}")

1094446080
proportion: 0.0001163; 1 in 8598


### Dragon Triplets

In [23]:
### One Dragon Triplet
# honors index 2 (non-dragon pair) or 3 (dragon pair)

winning_hands = 0
for hon_idx in range(2,4): # must include honor tiles
    hon_part = dragon_complete_ways.loc[hon_idx]

    sou_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']]
    for sou_idx in suited_complete_ways.index:
        sou_part = suited_complete_ways.loc[sou_idx]

        pin_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']-sou_part['n_tiles']]
        for pin_idx in pin_ways.index:
            pin_part = pin_ways.loc[pin_idx]
            
            man_ways = suited_complete_ways[suited_complete_ways['n_tiles'] == 14-hon_part['n_tiles']-sou_part['n_tiles']-pin_part['n_tiles']] 
            for man_idx in man_ways.index:
                man_part = man_ways.loc[man_idx]

                hand = sou_part + pin_part + man_part + hon_part

                if hand['n_pairs'] == 1: 
                    winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.1f}")

493083162000
proportion: 0.0524016; 1 in 19.1


In [24]:
### Two Dragon Triplets
# honors index 4

winning_hands = 0

hon_part = dragon_complete_ways.loc[4]
sou_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']]
for sou_idx in suited_complete_ways.index:
    sou_part = suited_complete_ways.loc[sou_idx]

    pin_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]
        
        man_ways = suited_complete_ways[suited_complete_ways['n_tiles'] == 14-hon_part['n_tiles']-sou_part['n_tiles']-pin_part['n_tiles']] 
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hand = sou_part + pin_part + man_part + hon_part

            if hand['n_pairs'] == 1: 
                winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.0f}")

5764832064
proportion: 0.0006126; 1 in 1632


In [25]:
### Small Three Dragons
# honors index 5

winning_hands = 0

hon_part = dragon_complete_ways.loc[5]
sou_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']]
for sou_idx in suited_complete_ways.index:
    sou_part = suited_complete_ways.loc[sou_idx]

    pin_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]
        
        man_ways = suited_complete_ways[suited_complete_ways['n_tiles'] == 14-hon_part['n_tiles']-sou_part['n_tiles']-pin_part['n_tiles']] 
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hand = sou_part + pin_part + man_part + hon_part

            if hand['n_pairs'] == 1: 
                winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.3e}; 1 in {total_winning_hands/winning_hands:.0f}")

258792192
proportion: 2.750e-05; 1 in 36360


In [26]:
### Large Three Dragons
# honors index 6

winning_hands = 0

hon_part = dragon_complete_ways.loc[6]
sou_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']]
for sou_idx in suited_complete_ways.index:
    sou_part = suited_complete_ways.loc[sou_idx]

    pin_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-hon_part['n_tiles']-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]
        
        man_ways = suited_complete_ways[suited_complete_ways['n_tiles'] == 14-hon_part['n_tiles']-sou_part['n_tiles']-pin_part['n_tiles']] 
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hand = sou_part + pin_part + man_part + hon_part

            if hand['n_pairs'] == 1: 
                winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.3e}; 1 in {total_winning_hands/winning_hands:.0f}")

13722624
proportion: 1.458e-06; 1 in 685707


### Single Numeric Suit

In [27]:
### Half Flush
# calculate for single suit; multiply by 3 for all suits

winning_hands = 0
for hon_idx in range(1,7): # must have some number of honors, standard hands
    sou_idx = 9-hon_idx
    winning_hands += suited_complete_ways.loc[sou_idx, 'n_ways'] * dragon_complete_ways.loc[hon_idx,'n_ways']
for hon_idx in range(1,4): # chiitoi hands
    hon_part = dragon_pairs_ways.loc[man_idx]
    
    sou_ways = suited_pairs_ways[suited_pairs_ways['n_tiles'] == 14-hon_part['n_tiles']]
    for sou_idx in sou_ways.index:
        sou_part = sou_ways.loc[sou_idx]

        winning_hands += sou_part['n_ways'] * man_part['n_ways']
winning_hands *= 3

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.0f}")

5462454474
proportion: 0.0005805; 1 in 1723


In [28]:
### Full Flush
# calculate for a single suit; multiply by 3 for all suits
winning_hands = suited_complete_ways.loc[9, 'n_ways'] + suited_pairs_ways.loc[12, 'n_ways']
winning_hands *= 3

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.0f}")

1336897596
proportion: 0.0001421; 1 in 7038


### Other Set Patterns

In [29]:
identical_sequences = [x+x for x in sequences[:-1]]

valid_groups = assemble_from_groups(identical_sequences, sets, sets, pairs)
iipeikou_complete = suited_complete.loc[valid_groups,:]
iipeikou_complete_ways = iipeikou_complete.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

valid_groups = assemble_from_groups(identical_sequences, identical_sequences, pairs)
ryanpeikou_complete = suited_complete.loc[valid_groups,:]
ryanpeikou_complete_ways = ryanpeikou_complete.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

In [30]:
### Twice Two Identical Sequences
# calculate for a single suit; multiply by 3 for all suits

# case 1: iipeikou in two different suits
ryanpeikou_hands  = iipeikou_complete_ways.loc[0, 'n_ways'] * iipeikou_complete_ways.loc[0, 'n_ways'] * (12 * 6)
ryanpeikou_hands += iipeikou_complete_ways.loc[1, 'n_ways'] * 2 * iipeikou_complete_ways.loc[0, 'n_ways']

# case 2: ryanpeikou in a single suit
ryanpeikou_hands += ryanpeikou_complete_ways.loc[0, 'n_ways'] * (21 * 6)
ryanpeikou_hands += ryanpeikou_complete_ways.loc[1, 'n_ways']

# all numeric suits
ryanpeikou_hands *= 3 

print(ryanpeikou_hands)
print(f"proportion: {ryanpeikou_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/ryanpeikou_hands:.0f}")

1119645018
proportion: 0.0001190; 1 in 8404


In [31]:
### Two Identical Sequences
# calculate for a single suit; multiply by 3 for all suits

winning_hands = 0
for sou_idx in iipeikou_complete_ways.index:
    sou_part = iipeikou_complete_ways.loc[sou_idx]

    pin_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]

        man_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']-pin_part['n_tiles']]
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hon_ways = dragon_complete_ways[dragon_complete_ways['n_tiles'] == 14-sou_part['n_tiles']-pin_part['n_tiles']-man_part['n_tiles']]
            for hon_idx in hon_ways.index:
                hon_part = hon_ways.loc[hon_idx]
                hand = sou_part + pin_part + man_part + hon_part

                if hand['n_pairs'] == 1:
                    winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

winning_hands *= 3 # iipeikou in a different numeric suit
winning_hands -= ryanpeikou_hands # exclude "Twice Two Identical Sequences" hands

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.1f}")

356952436224
proportion: 0.0379345; 1 in 26.4


In [32]:
### Full Straight
# calculate for a single suit; multiply by 3 for all suits

valid_groups = assemble_from_groups([int_to_vector(123456789)], sets, pairs)
ittsu_complete = suited_complete.loc[valid_groups,:]
ittsu_complete_ways = ittsu_complete.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

winning_hands = 0
for sou_idx in ittsu_complete_ways.index:
    sou_part = ittsu_complete_ways.loc[sou_idx]

    pin_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]

        man_ways = suited_complete_ways[suited_complete_ways['n_tiles'] <= 14-sou_part['n_tiles']-pin_part['n_tiles']]
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hon_ways = dragon_complete_ways[dragon_complete_ways['n_tiles'] == 14-sou_part['n_tiles']-pin_part['n_tiles']-man_part['n_tiles']]
            for hon_idx in hon_ways.index:
                hon_part = hon_ways.loc[hon_idx]
                hand = sou_part + pin_part + man_part + hon_part

                if hand['n_pairs'] == 1:
                    winning_hands += sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']

winning_hands *= 3 # iipeikou in a different numeric suit

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.1f}")

95777832960
proportion: 0.0101786; 1 in 98.2


In [33]:
### Three Similar Sequences
# calculate for a single suit; multiply by 3 for all suits

winning_hands = 0
for sequence in sequences[:-1]: # different sequences have different allowances
    valid_groups = assemble_from_groups([sequence], sets, pairs)
    doujun_single = suited_complete.loc[valid_groups,:]
    doujun_single_ways = doujun_single.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

    # remaining set and pair are honors
    winning_hands += doujun_single_ways.loc[0,'n_ways'] ** 3 * dragon_complete_ways.loc[3,'n_ways']

    # one dragon triplet, one numeric pair
    winning_hands += doujun_single_ways.loc[1,'n_ways'] * doujun_single_ways.loc[0,'n_ways'] ** 2 * dragon_complete_ways.loc[2,'n_ways']

    # one dragon pair, one additional numeric set
    winning_hands += doujun_single_ways.loc[2,'n_ways'] * doujun_single_ways.loc[0,'n_ways'] ** 2 * dragon_complete_ways.loc[1,'n_ways']

    # remaining set and pair are in different numeric suits
    winning_hands += 2 * doujun_single_ways.loc[2,'n_ways'] * doujun_single_ways.loc[1,'n_ways'] * doujun_single_ways.loc[0,'n_ways']

    # remaining set and pair are in a single numeric suit
    winning_hands += doujun_single_ways.loc[3,'n_ways'] * doujun_single_ways.loc[0,'n_ways'] ** 2

winning_hands *= 3 # rotate the focus numeric suit

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.7f}; 1 in {total_winning_hands/winning_hands:.1f}")

178559238144
proportion: 0.0189761; 1 in 52.7


In [34]:
### Three Similar Triplets
# calculate for a single suit; multiply by 3 for all suits

winning_hands = 0
for triplet in triplets[:-1]: # different triplets have different allowances
    valid_groups = assemble_from_groups([triplet], sets, pairs)
    doukou_single = suited_complete.loc[valid_groups,:]
    doukou_single_ways = doukou_single.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

    # remaining set and pair are honors
    winning_hands += doukou_single_ways.loc[0,'n_ways'] ** 3 * dragon_complete_ways.loc[3,'n_ways']

    # one dragon triplet, one numeric pair
    winning_hands += doukou_single_ways.loc[1,'n_ways'] * doukou_single_ways.loc[0,'n_ways'] ** 2 * dragon_complete_ways.loc[2,'n_ways']

    # one dragon pair, one additional numeric set
    winning_hands += doukou_single_ways.loc[2,'n_ways'] * doukou_single_ways.loc[0,'n_ways'] ** 2 * dragon_complete_ways.loc[1,'n_ways']

    # remaining set and pair are in different numeric suits
    winning_hands += 2 * doukou_single_ways.loc[2,'n_ways'] * doukou_single_ways.loc[1,'n_ways'] * doukou_single_ways.loc[0,'n_ways']

    # remaining set and pair are in a single numeric suit
    winning_hands += doukou_single_ways.loc[3,'n_ways'] * doukou_single_ways.loc[0,'n_ways'] ** 2

winning_hands *= 3 # rotate the focus numeric suit

print(winning_hands)
print(f"proportion: {winning_hands/total_winning_hands:0.3e}; 1 in {total_winning_hands/winning_hands:.0f}")

88187904
proportion: 9.372e-06; 1 in 106701


## Shanten Calculations

- What is the shanten distribution across all thirteen-tile hands, and what is the average shanten count?

In [75]:
suited_df['lone_tiles'] = suited_df['tile_vector'].apply(lambda x: (string_to_vector(x) == 1).sum())
suited_ways = suited_df.groupby(['n_tiles', 'n_sets', 'n_blocks', 'n_pairs', 'max_pairs', 'lone_tiles']).agg({'n_ways': sum}).reset_index()
suited_ways = suited_ways[suited_ways['n_tiles'] <= 13]

dragon_df['lone_tiles'] = dragon_df['tile_vector'].apply(lambda x: (string_to_vector(x) == 1).sum())
dragon_ways = dragon_df.groupby(['n_tiles', 'n_triplets', 'n_pairs', 'lone_tiles']).agg({'n_ways': sum}).reset_index()
dragon_ways = dragon_ways.rename(columns={'n_triplets':'n_sets'})
dragon_ways['n_blocks']  = dragon_ways['n_pairs']
dragon_ways['max_pairs'] = dragon_ways['n_pairs'] + dragon_ways['n_sets']

In [77]:
shanten_ways = np.zeros(7,dtype=np.int64)
shanten_matrix = np.zeros([9,7],dtype=np.int64)

for sou_idx in suited_ways.index:
    sou_part = suited_ways.loc[sou_idx]
    
    pin_ways = suited_ways[suited_ways['n_tiles'] <= 13-sou_part['n_tiles']]
    for pin_idx in pin_ways.index:
        pin_part = pin_ways.loc[pin_idx]

        man_ways = suited_ways[suited_ways['n_tiles'] <= 13-sou_part['n_tiles']-pin_part['n_tiles']]
        for man_idx in man_ways.index:
            man_part = man_ways.loc[man_idx]

            hon_ways = dragon_ways[dragon_ways['n_tiles'] == 13-sou_part['n_tiles']-pin_part['n_tiles']-man_part['n_tiles']]
            for hon_idx in hon_ways.index:
                hon_part = hon_ways.loc[hon_idx]
                hand = sou_part + pin_part + man_part + hon_part

                # calculate shanten
                has_pair = min(hand['n_pairs'], 1)
                standard_shanten = 8 - 2 * hand['n_sets'] - has_pair - min(hand['n_blocks']-has_pair, 4-hand['n_sets'])
                chiitoi_base = 6 - hand['max_pairs']
                chiitoi_shanten = chiitoi_base + max(1 + chiitoi_base - hand['lone_tiles'], 0)
                
                shanten = min(standard_shanten, chiitoi_shanten)
                n_ways = sou_part['n_ways'] * pin_part['n_ways'] * man_part['n_ways'] * hon_part['n_ways']
                shanten_ways[shanten] += n_ways
                shanten_matrix[standard_shanten, chiitoi_shanten] += n_ways

In [78]:
print(f'hands by shanten (in trillions):')
print(shanten_ways / 1e12)
print('')
print(f'tenpai chance: {shanten_ways[0] / shanten_ways.sum():0.7f}; 1 in {shanten_ways.sum() / shanten_ways[0]:0.0f}')
print(f'average shanten: {(shanten_ways * np.arange(7)).sum() / shanten_ways.sum():0.2f}')
print('')
print('probabilities and ratios')
print(shanten_ways / shanten_ways.sum())
print(shanten_ways.sum() / shanten_ways)

hands by shanten (in trillions):
[2.75102705e+01 1.66443081e+03 1.79558568e+04 4.27105356e+04
 2.24796655e+04 2.70162410e+03 4.72100792e+01]

tenpai chance: 0.0003141; 1 in 3184
average shanten: 3.08

probabilities and ratios
[3.14091394e-04 1.90032080e-02 2.05006348e-01 4.87636487e-01
 2.56655763e-01 3.08450940e-02 5.39008860e-04]
[3.18378670e+03 5.26226939e+01 4.87789773e+00 2.05070791e+00
 3.89626942e+00 3.24200666e+01 1.85525707e+03]


In [79]:
print(f'standard vs chiitoi shanten matrix (in billions)')
print(np.round(shanten_matrix / 1e9, 0).astype(np.int64))

standard vs chiitoi shanten matrix (in billions)
[[       5      132      906     3439     8482    10042     1851]
 [     243     5626    56470   251450   554370   518479   102835]
 [     167    50738   641209  2996508  6049357  5052897  1139334]
 [    2244   124462  1611190  7531126 15035622 12507096  3396173]
 [       0        0   465362  3763008  9215002  8479164  2799441]
 [       0        0        0   477510  1890822  1992739   550102]
 [       0        0        0        0    95236   156095    46304]
 [       0        0        0        0        0     2688      906]
 [       0        0        0        0        0        0        0]]


#### Chiitoi Shanten Notes

Sometimes, shanten with respect to chiitoitsu is not as simple as taking 6 minus the number of pairs.

In [80]:
shanten_matrix.sum(axis=0)

array([    2659503974400,   180957082924800,  2775136990953600,
       15023042076107520, 32848890769356480, 28719199355041440,
        8036947486748880], dtype=int64)

In [54]:
# For shanten 0, we require six pairs and a unique seventh tile.
# If the last tile matches one of the six pairs, it is counted as shanten 1!

math.comb(30,6) * 24 * 6 ** 6 * 4

2659503974400

In [56]:
# On the other end, you might expect shanten 6 to simply be thirteen
# unique tiles.

math.comb(30,13) * 4 ** 13

8036947486310400

In [82]:
# However, there are additional edge cases where there aren't enough unique, unpaired tiles
# from which to form additional pairs! In these cases, additional shanten are gained due
# to the need to replace tiles from triplets and quads with new tiles.
# For example: "AAAA BBBB CCCC D E" has three pairs and two lone tiles, but is considered
# 6-shanten with respect to chiitoi, since the excess "AA BB CC" must be shed first. (Its
# overall shanten will be much lower as a standard hand.)

8036947486748880 - 8036947486310400

438480